In [115]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
import json
import pandas as pd
import plotly

ModuleNotFoundError: No module named 'plotly'

In [79]:
cid = 'bfa6b664138747c1be0271cffb6dfe7b'
secret = '39a4198c3d4a4616bf32eff8bd132088'

In [80]:
clientCreds = SpotifyClientCredentials(client_id=cid, client_secret=secret)
#client_credentials_manager=clientCreds, 
scope = 'playlist-read-private'
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=cid, client_secret=secret, redirect_uri='http://localhost/'))

In [81]:


sp.current_user()


{'display_name': 'tjkirchmann',
 'external_urls': {'spotify': 'https://open.spotify.com/user/tjkirchmann'},
 'followers': {'href': None, 'total': 6},
 'href': 'https://api.spotify.com/v1/users/tjkirchmann',
 'id': 'tjkirchmann',
 'images': [{'height': None,
   'url': 'https://i.scdn.co/image/ab6775700000ee85e7f5401a0cb01313037a9f28',
   'width': None}],
 'type': 'user',
 'uri': 'spotify:user:tjkirchmann'}

In [82]:
current_playlists = sp.current_user_playlists(limit=50)

pl_id = [[],[]]
for i, item in enumerate(current_playlists['items']):
    pl_id[0].append(item['id'])
    pl_id[1].append(item['name'])

for i in range(len(pl_id[0])):
    pl_id[0][i] = 'spotify:playlist:' + pl_id[0][i]
    print(pl_id[0][i] + ": ||| : " + pl_id[1][i])

spotify:playlist:3pfsccuQqWtIPArcOPKVAz: ||| : is it time?!
spotify:playlist:07AOcL67ldMQldSxpcSUT7: ||| : My Playlist #44
spotify:playlist:2WDJr1pPG81e2Hfxa7O4Aq: ||| : Set
spotify:playlist:08xzoHhuesCvNkRZQ8ouu7: ||| : Bangers W Few Words
spotify:playlist:5R8F09qKoeyqLLoCCKLFe4: ||| : Good raps
spotify:playlist:64SMwJvKBbHO0Qw9DXpMS4: ||| : oLD sCHOOL
spotify:playlist:4At80e49hdTUxcoQOw9oTe: ||| : Anti-Chimera Era
spotify:playlist:669yyRrmnl9mAjjVZwpAxB: ||| : FREDDDIEE
spotify:playlist:1f1JwwzmEDx2EmHiZWaze3: ||| : Street Beats
spotify:playlist:528CFdEt5eHusIzvjSDE7i: ||| : The Finest Fruit in the Dance Factory
spotify:playlist:3kKtova92yKF7VTowphRkA: ||| : LoFi
spotify:playlist:5CVaeyTv1LfG1SeX303SKd: ||| : Day Drink
spotify:playlist:1zhPXdN3IyPwqSj861SSET: ||| : Bounce
spotify:playlist:5pKWsxmNg9RCPJ6HskZgbt: ||| : Travi and Co.
spotify:playlist:7GORFQOooOr83HKBf65Bpa: ||| : Karaoke Options
spotify:playlist:78iu1zOeXvjTHgbDA40Zei: ||| : Split//2
spotify:playlist:2ysqamasZASZvy04If

In [88]:
upper = sp.playlist_items('spotify:playlist:6aqeNH2BHDksU8WvX44wJ0', fields='items.track.href')
tracks = []
#print(upper)
for item in upper['items']:
    track = item['track']
    href = track['href']
    songId = href.replace('https://api.spotify.com/v1/tracks/','')
    tracks.append(songId)    

In [112]:
playlistFeatures = pd.DataFrame(columns=['name','id','artist','artist_id','release_date','popularity','explicit','duration_ms',
                                        'danceability','enery','key','loudness','mode','speechiness','acousticness','instrumentalness',
                                        'liveness','valence','tempo','type','time_signature'])
features = sp.audio_features(tracks=tracks)
for i in range(len(features)):
    #Track Info init from a get on the track id --- Track Features from the call above that returns all the features in the playlist
    trackInfo = sp.track(tracks[i])
    trackFeatures = features[i]
    #print(trackFeatures)
    #trackFeatures = trackFeatures[0]
    #Get multiple artists if needed, else return the one artist
    artistData = trackInfo.get("artists")
    if len(artistData) != 1:
        artists = [artist.get("name") for artist in artistData]
        artistsID = [artist.get("id") for artist in artistData]
    else:
        artists = artistData[0].get("name")
        artistsID = artistData[0].get("id")

    newRow = {
        'name': trackInfo.get("name"),
        'id': tracks[i],
        'artist': artists,
        'artist_id': artistsID,
        'release_date': trackInfo.get("album", "no_album").get("release_date", "no_release"),
        'popularity': trackInfo.get("popularity"),
        'explicit': trackInfo.get("explicit"),
        'duration_ms': trackInfo.get("duration_ms"),
        'danceability': trackFeatures.get("danceability"),
        'energy': trackFeatures.get("energy"),
        'key': trackFeatures.get("key"),
        'loudness': trackFeatures.get("loudness"),
        'mode': trackFeatures.get("mode"),
        'speechiness': trackFeatures.get("speechiness"),
        'acousticness': trackFeatures.get("acousticness"),
        'instrumentalness': trackFeatures.get("instrumentalness"),
        'liveness': trackFeatures.get("liveness"),
        'valence': trackFeatures.get("valence"),
        'tempo': trackFeatures.get("tempo"),
        'type': trackFeatures.get("type"),
        'time_signature': trackFeatures.get("time_signature")
    }
    playlistFeatures = playlistFeatures.append(newRow, ignore_index=True)


{'danceability': 0.756, 'energy': 0.609, 'key': 3, 'loudness': -7.927, 'mode': 1, 'speechiness': 0.0544, 'acousticness': 0.217, 'instrumentalness': 0.00474, 'liveness': 0.096, 'valence': 0.537, 'tempo': 117.023, 'type': 'audio_features', 'id': '5EYi2rH4LYs6M21ZLOyQTx', 'uri': 'spotify:track:5EYi2rH4LYs6M21ZLOyQTx', 'track_href': 'https://api.spotify.com/v1/tracks/5EYi2rH4LYs6M21ZLOyQTx', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5EYi2rH4LYs6M21ZLOyQTx', 'duration_ms': 155720, 'time_signature': 4}
{'danceability': 0.76, 'energy': 0.444, 'key': 0, 'loudness': -8.976, 'mode': 0, 'speechiness': 0.386, 'acousticness': 0.431, 'instrumentalness': 1.25e-06, 'liveness': 0.106, 'valence': 0.66, 'tempo': 144.15, 'type': 'audio_features', 'id': '37b1KAbfOZeBzeMB0LGO3g', 'uri': 'spotify:track:37b1KAbfOZeBzeMB0LGO3g', 'track_href': 'https://api.spotify.com/v1/tracks/37b1KAbfOZeBzeMB0LGO3g', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/37b1KAbfOZeBzeMB0LGO3g', 'duration

{'danceability': 0.494, 'energy': 0.587, 'key': 9, 'loudness': -6.478, 'mode': 1, 'speechiness': 0.171, 'acousticness': 0.547, 'instrumentalness': 0, 'liveness': 0.28, 'valence': 0.214, 'tempo': 161.897, 'type': 'audio_features', 'id': '2agURy6Jj70naGfDcrKkwY', 'uri': 'spotify:track:2agURy6Jj70naGfDcrKkwY', 'track_href': 'https://api.spotify.com/v1/tracks/2agURy6Jj70naGfDcrKkwY', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2agURy6Jj70naGfDcrKkwY', 'duration_ms': 253698, 'time_signature': 4}
{'danceability': 0.431, 'energy': 0.773, 'key': 2, 'loudness': -5.002, 'mode': 1, 'speechiness': 0.0636, 'acousticness': 0.0264, 'instrumentalness': 0, 'liveness': 0.348, 'valence': 0.504, 'tempo': 111.387, 'type': 'audio_features', 'id': '2RGe02P8xxSF9syj0ltPjX', 'uri': 'spotify:track:2RGe02P8xxSF9syj0ltPjX', 'track_href': 'https://api.spotify.com/v1/tracks/2RGe02P8xxSF9syj0ltPjX', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2RGe02P8xxSF9syj0ltPjX', 'duration_ms': 2709

{'danceability': 0.64, 'energy': 0.755, 'key': 1, 'loudness': -4.077, 'mode': 0, 'speechiness': 0.0287, 'acousticness': 0.0138, 'instrumentalness': 0.0208, 'liveness': 0.12, 'valence': 0.744, 'tempo': 116.883, 'type': 'audio_features', 'id': '4g3Ax56IslQkI6XVfYKVc5', 'uri': 'spotify:track:4g3Ax56IslQkI6XVfYKVc5', 'track_href': 'https://api.spotify.com/v1/tracks/4g3Ax56IslQkI6XVfYKVc5', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4g3Ax56IslQkI6XVfYKVc5', 'duration_ms': 216319, 'time_signature': 4}
{'danceability': 0.335, 'energy': 0.818, 'key': 8, 'loudness': -5.551, 'mode': 1, 'speechiness': 0.052, 'acousticness': 0.00546, 'instrumentalness': 0.00246, 'liveness': 0.204, 'valence': 0.661, 'tempo': 177.491, 'type': 'audio_features', 'id': '2HLQkvx1WUQgzSnhaulYlX', 'uri': 'spotify:track:2HLQkvx1WUQgzSnhaulYlX', 'track_href': 'https://api.spotify.com/v1/tracks/2HLQkvx1WUQgzSnhaulYlX', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2HLQkvx1WUQgzSnhaulYlX', 'durati

{'danceability': 0.505, 'energy': 0.489, 'key': 11, 'loudness': -9.943, 'mode': 1, 'speechiness': 0.031, 'acousticness': 0.229, 'instrumentalness': 0.792, 'liveness': 0.359, 'valence': 0.772, 'tempo': 77.298, 'type': 'audio_features', 'id': '0EXdNsNOxZH1TLzBYUgY55', 'uri': 'spotify:track:0EXdNsNOxZH1TLzBYUgY55', 'track_href': 'https://api.spotify.com/v1/tracks/0EXdNsNOxZH1TLzBYUgY55', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0EXdNsNOxZH1TLzBYUgY55', 'duration_ms': 190956, 'time_signature': 4}
{'danceability': 0.719, 'energy': 0.369, 'key': 1, 'loudness': -9.12, 'mode': 0, 'speechiness': 0.193, 'acousticness': 0.472, 'instrumentalness': 0.0572, 'liveness': 0.102, 'valence': 0.364, 'tempo': 150.076, 'type': 'audio_features', 'id': '5dl8x4xLEzZC0f927WBTUG', 'uri': 'spotify:track:5dl8x4xLEzZC0f927WBTUG', 'track_href': 'https://api.spotify.com/v1/tracks/5dl8x4xLEzZC0f927WBTUG', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5dl8x4xLEzZC0f927WBTUG', 'duration_ms

In [114]:
print(playlistFeatures)
playlistFeatures.to_csv('Indie-ish.csv')

                                    name                      id  \
0                         Loving Is Easy  5EYi2rH4LYs6M21ZLOyQTx   
1                                Napkins  37b1KAbfOZeBzeMB0LGO3g   
2                               Red Eyes  71jGGLe5VtEHjIk5dU2W3S   
3                       Only in the West  4A3yBa4C3Zy6UAGxIyv1XD   
4                            Pretty Girl  4Pv7OY4tfCA8KIdtlkANOS   
5                               Dark Red  37y7iDayfwm3WXn5BiAoRk   
6     Diddy Bop (feat. Cam O'bi & Raury)  6JvfBzqZmSiEG5MjM7OcSY   
7                             Why Not Me  1gewwJHPkvysxLEGgOKQuB   
8                                 Sunday  3Plhh6DXBtxPF59hxZVwBO   
9                              Tired Boy  4SkTfsjWUy2rp40d2Ayfmv   
10                      Warm (feat. Mia)  3kBofOTKMUZ62a311eUwvx   
11           B Cool (feat. Melanie Faye)  6EErYXgsGBMkwpITHuKuCI   
12        Baby I'm Yours - feat. Irfrane  7oGwQOTkMB9Sk3DIKJLd5F   
13            Go With It (feat. Chromeo)  1xF1lC